In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
from scipy.stats import mode
import string
import matplotlib.pyplot as plt
import seaborn as sns 
import warnings
warnings.filterwarnings("ignore")
import gc
import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
# Memory saving function credit to https://www.kaggle.com/gemartin/load-data-reduce-memory-usage
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type to reduce memory usage."""
    start_mem = df.memory_usage().sum() / 1024**2
    
    for col in df.columns:
        col_type = df[col].dtype
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB --> {:.2f} MB (Decreased by {:.1f}%)'.format(
        start_mem, end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [ ]:
train=pd.read_csv("../input/train_V2.csv")
train=reduce_mem_usage(train)#[:1000000])
#test=pd.read_csv("../input/test_V2.csv")
#test=reduce_mem_usage(test)#[:50000])
train.head()

In [ ]:
#test.head()

In [ ]:
#print(train["Id"].isin(test["Id"]).any(),train["matchId"].isin(test["matchId"]).any())
#### thank god no overlap between test and train

In [ ]:
gc.collect()

# **data analysis**

In [ ]:
print("The average person kills {:.4f} players, 99% of people have {} kills or less, while the most kills ever recorded is {}.".format(train['kills'].mean(),train['kills'].quantile(0.99), train['kills'].max()))

# **Feature engineer**

In [ ]:
train['playersJoined'] = train.groupby('matchId')['matchId'].transform('count')

In [ ]:
train['MatchMeanKill'] = train.groupby('matchId')['kills'].transform('mean').round(2)
train['MatchMeanDamage'] = train.groupby('matchId')['damageDealt'].transform('mean').round()
train['GroupMeanKill'] = train.groupby('groupId')['kills'].transform('mean').round(2)
train['GroupMeanDamage'] = train.groupby('groupId')['damageDealt'].transform('mean').round()

In [ ]:
train['healsAndBoosts'] = train['heals']+train['boosts']
train['totalDistance'] = train['walkDistance']+train['rideDistance']+train['swimDistance']
train['MatchMeanheal'] = train.groupby('matchId')['healsAndBoosts'].transform('mean').round(2)
train['MatchMeanDistance'] = train.groupby('matchId')['totalDistance'].transform('mean').round()

In [ ]:
train['healsperc'] = train['healsAndBoosts']/train['MatchMeanheal']
train['healsperc']= train['healsperc'].round(2)
train['Distanceperc'] = train['totalDistance']/(train['MatchMeanDistance']+1)
train['Distanceperc']= train['Distanceperc'].round(2)
train[['playersJoined', 'MatchMeanheal', 'MatchMeanDistance', 'healsAndBoosts', 'totalDistance','healsperc','Distanceperc']][10:20]

In [ ]:
train['NevigateTimeperc'] = (train['walkDistance']/5+train['rideDistance']/27+train['swimDistance']/2)/train['matchDuration']

In [ ]:
train.head(10)

In [ ]:
print(train['matchType'].value_counts())

### seperate out event mode

In [ ]:
crashrows=train.query('matchType in ["crashtpp","crashfpp"]').index
crashdata = train.loc[crashrows]
train=train.drop(crashrows)

In [ ]:
flarerows=train.query('matchType in ["flaretpp","flarefpp"]').index
flaredata = train.loc[flarerows]
train=train.drop(flarerows)

In [ ]:
print(train['matchType'].value_counts())
print(crashdata['matchType'].value_counts())
print(flaredata['matchType'].value_counts())

In [ ]:
mapper = lambda x: 1 if ('solo' in x) else 2 if ('duo' in x) else 4
train['matchTyper'] = train['matchType'].map(mapper)
print(train['matchTyper'].value_counts())

In [ ]:
train['groupplayer'] = train.groupby(['matchId','groupId'])['groupId'].transform('count')
print(train['groupplayer'].value_counts())

In [ ]:
train[train['groupplayer']>70].head(10)

In [ ]:
train['allin'] = train['groupplayer']/train['matchTyper']
sns.distplot(train['allin'])
plt.show()

In [ ]:
abnormrows=train.query('allin>1').index
abnormdata = train.loc[abnormrows]
train=train.drop(abnormrows)
print(train['groupplayer'].value_counts())
print(abnormdata['groupplayer'].value_counts())

In [ ]:
sns.distplot(train['allin'])
plt.show()

In [ ]:
print("The average person kills {:.4f} players, 99% of people have {} kills or less, while the most kills ever recorded is {}.".format(train['kills'].mean(),train['kills'].quantile(0.99), train['kills'].max()))
data = train.copy()
data.loc[data['kills'] > data['kills'].quantile(0.99)] = '8+'
sns.countplot(data['kills'].astype('str').sort_values())
plt.title("Kill Count")
plt.show()
del data

In [ ]:
sns.jointplot(x="winPlacePerc", y="kills", data=train, color="r")
plt.show()

In [ ]:
sns.jointplot(x="winPlacePerc", y="GroupMeanKill", data=train, color="r")
plt.show()

In [ ]:
kills = train.copy()
kills['killsCategories'] = pd.cut(kills['kills'], [-1, 0, 2, 5, 10, 65], labels=['0_kills','1-2_kills', '3-5_kills', '6-10_kills', '10+_kills'])
sns.boxplot(x="killsCategories", y="winPlacePerc", data=kills)
plt.show()
del kills

In [ ]:
f,ax = plt.subplots(figsize=(15, 15))
sns.heatmap(train.corr(), annot=True, linewidths=.5, fmt= '.1f',ax=ax)
plt.show()

In [ ]:
k = 5 #number of variables for heatmap
cols = train.corr().nlargest(k, 'winPlacePerc')['winPlacePerc'].index
sns.set(font_scale=1.25)
hm = sns.heatmap(train[cols].corr(), cbar=True, annot=True, square=True, fmt='.2f', annot_kws={'size': 10}, yticklabels=cols.values, xticklabels=cols.values)
plt.show()

In [ ]:
sns.set()
sns.jointplot(x="winPlacePerc", y="walkDistance", data=train, color="r")
plt.show()

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(12, 4))
# there are two types of maps
train['matchDuration'].hist(bins=50, ax=ax[0])
train.query('matchDuration >= 1400 & matchDuration <= 1800')['matchDuration'].hist(bins=50, ax=ax[1])

In [ ]:
train['killsperc'] = train['GroupMeanKill']/train['MatchMeanKill']
train['killsperc']= train['killsperc'].round(2)
train['damageperc'] = train['GroupMeanDamage']/train['MatchMeanDamage']
train['damageperc']= train['damageperc'].round(2)
train[['playersJoined', 'kills', 'damageDealt','killsperc','damageperc']][10:20]

In [ ]:
sns.jointplot(x="winPlacePerc", y="killsperc", data=train, color="r")
plt.show()

In [ ]:
sns.jointplot(x="winPlacePerc", y="damageperc", data=train, color="r")
plt.show()

In [ ]:
sns.jointplot(x="winPlacePerc", y="healsperc", data=train, color="r")
plt.show()

In [ ]:
sns.jointplot(x="winPlacePerc", y="Distanceperc", data=train, color="r")
plt.show()

In [ ]:
sns.jointplot(x="winPlacePerc", y="totalDistance", data=train, color="r")
plt.show()

In [ ]:
sns.jointplot(x="winPlacePerc", y="NevigateTimeperc", data=train, color="r")
plt.show()

In [ ]:
train[["Distanceperc",'NevigateTimeperc']].corr()

In [ ]:
X_train = train[train['winPlacePerc'].notnull()].reset_index(drop=True)
#X_test = all_data[all_data['winPlacePerc'].isnull()].drop(['winPlacePerc'], axis=1).reset_index(drop=True)
#del all_data
gc.collect()

Y_train = X_train.pop('winPlacePerc')
#X_test_grp = X_test[['matchId','groupId']].copy()
#train_matchId = X_train['matchId']

# drop matchId,groupId
X_train.drop(['Id','matchId','groupId','matchType'], axis=1, inplace=True)
#X_test.drop(['matchId','groupId'], axis=1, inplace=True)

print(X_train.shape)#, X_test.shape)

In [ ]:
from sklearn.model_selection import GroupKFold
from sklearn.preprocessing import minmax_scale
import lightgbm as lgb

params={'learning_rate': 0.05,
        'objective':'mae',
        'metric':'mae',
        'num_leaves': 31,
        'verbose': 0,
        'random_state':42,
        'bagging_fraction': 0.7,
        'feature_fraction': 0.7
       }
mts = list()
fis = list()
pred = np.zeros(X_train.shape[0])
for mt in X_train['matchTyper'].unique():
    idx = X_train[X_train['matchTyper'] == mt].index
    reg = lgb.LGBMRegressor(**params, n_estimators=3000)
    reg.fit(X_train.loc[idx], Y_train.loc[idx])

    idx = X_test[X_test['matchTypeCat'] == mt].index
    pred[idx] = reg.predict(X_test.loc[idx], num_iteration=reg.best_iteration_)
    mts.append(mt)
    fis.append(reg.feature_importances_)

In [ ]:
import lightgbm as lgb
gbm = lgb.LGBMRegressor(objective='regression',num_leaves=100,learning_rate=0.05,n_estimators=1500)
x_test, y_test=X_train[3000000:], Y_train[3000000:]

gbm.fit(X_train[:3000000], Y_train[:3000000],
    eval_set=[(x_test, y_test)],
    eval_metric='l2_root',
    early_stopping_rounds=10)
y_pred = gbm.predict(x_test, num_iteration=gbm.best_iteration_)
accuracy = round(gbm.score(x_train, y_train)*100,2)
mse = mean_squared_error(y_test,y_pred)
rmse = np.sqrt(mse)
print(accuracy,rmse)
